In [11]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Домашняя работа: ансамбли

Эта домашняя работа является идейным продолжением предыдущей. В данной работе вам будет необходимо поэкспериментировать с различными методами ансамблирования и проверить, какие из них работают лучше. Пайплайн предобработки данных можно взять полностью из предыдущей работы.

Требования к домашней работе:
- Во всех графиках (если вы их строите) должны быть подписи через title, legend, etc.
- Во время обучения моделей проверяйте, что у вас не текут данные. Обычно это позитивно влияет на качество модели на тесте, но негативно влияет на оценку 🌚
- Если вы сдаете работу в Google Colaboratory, убедитесь, что ваша тетрадка доступна по ссылке. Если в итоге по каким-то причинам тетрадка не будет открываться у преподавателя, задание не будет засчитано
- Использование мемов допускается. Если задания дались тяжело, можно дополнительно приложить какой-нибудь постироничный мем про ваши страдания во время выполнения данной домашней работы. За мемы с использованием нецензурной лексики баллы будут снижены.

# Загрузка и подготовка данных (1 балл)

В этой секции предлагается прогнать предобработку данных из прошлой тетрадки заново и сохранить получившийся датасет в формате csv.

Если вы **не хотите заморачиваться**, то просто скопируйте код с предобработкой ниже.

В противном случае в старой тетрадке:
1. Отдельно выполните предобработку (`fit_transform`) тренировочной части данных
2. Добавьте колонку `split` к датафрейму с обучающей выборкой, в этой колонке проставьте значение `train` для всех объектов
3. Затем примените **только** предобработку (`transform`) к тестовой части данных
4. Добавьте колонку `split` к тестовой выборке, в этой колонке проставьте значение `test` для всех объектов
5. Объедините два датафрейма в один при помощи функции `pd.concat`
6. Сохраните получившийся датафрейм при помощи функции `to_csv`, не забудьте передать аргумент `index=False`

Получившийся файл сохраните отдельно и используйте в этой домашней работе. Для разбиения датасета на обучающую и тестовую части вместо функции `train_test_split` можете применять колонку `split`.

In [311]:
path = "C:/Users/Дмитрий/Desktop/hw_10/ml-fall22/10. Ensembles/prepaired_data.csv"
prepaired_data = pd.read_csv(path)
X_train, X_test = prepaired_data.query('split=="train"'), prepaired_data.query('split=="test"')
y_train, y_test = prepaired_data.query('split=="train"').target, prepaired_data.query('split=="test"').target

In [312]:
from sklearn import set_config
set_config(transform_output="pandas")

trans = ColumnTransformer(
    [('num', StandardScaler(with_mean=False), [1,2,3,4]),
     ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), [0,5,6,7,8,9])])

X_train= trans.fit_transform(X_train)
X_test = trans.transform(X_test)

Важно: во всех разделах ниже задачу регрессии важно оценивать не только при помощи `MSE`, но и при помощи `r2_score`. Если вы хотите перебрать какой-либо гиперпараметр, не забывайте оценивать то, насколько сильно переобучается модель и как меняется каждый из параметров в процессе обучения.

In [ ]:
from sklearn.model_selection import train_test_split

# Стекинг (максимум 3 балла)

Решите задачу, используя разные комбинации базовых моделей. В качестве базовой модели обязательно попробуйте линейную регрессию, дерево и SVM для регрессии.

Какой набор моделей дает лучший результат? Попробуйте улучшить его, перебрав несколько гиперпараметров (как у базовой модели, так и у ансамбля).

## Простой стекинг своими руками (2 балла)

In [343]:
class StackingRegressionSolver:
    def __init__(self, base_estimators: list, meta_estimator):
        self._base_estimators = base_estimators
        self._meta_estimator = meta_estimator

    def _fit_base(self, X: pd.DataFrame, y: pd.Series) -> None:
        for estimator in self._base_estimators:
            estimator.fit(X, y)

    def _predict_base(self, X: pd.DataFrame) -> pd.DataFrame:
        return np.array([estimator.predict(X) for estimator in self._base_estimators]).T

    def fit(self, X: pd.DataFrame, y: pd.Series):
        self._fit_base(X, y)
        meta_features = self._predict_base(X)
        self._meta_estimator.fit(meta_features, y)
        return self

    def predict(self, X: pd.DataFrame) -> pd.Series:
        meta_features = self._predict_base(X)
        return self._meta_estimator.predict(meta_features)

In [344]:
from sklearn.metrics import mean_squared_error as mse, r2_score
def draw_metrics(y_pred_test, y_pred_train, y_test, y_train):
    print('Test\nMSE & RMSE & R2')
    print(mse(y_test, y_pred_test), mse(y_test, y_pred_test, squared=False), r2_score(y_test, y_pred_test))
    print('Train\nMSE & RMSE & R2')
    print(mse(y_train, y_pred_train), mse(y_train, y_pred_train, squared=False), r2_score(y_train, y_pred_train))


In [345]:
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

linreg = LinearRegression()
svm = LinearSVR()
tree = DecisionTreeRegressor()

stacking_regressor = StackingRegressionSolver([svm, tree], linreg)
stacking_regressor.fit(X_train, y_train)
y_pred_test = stacking_regressor.predict(X_test)
y_pred_train = stacking_regressor.predict(X_train)
draw_metrics(y_pred_test, y_pred_train, y_test, y_train)

c:\Users\Дмитрий\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Test
MSE & RMSE & R2
3890.2547370070424 62.37190663277051 0.023805188373941855
Train
MSE & RMSE & R2
91.8492922542257 9.583803642303284 0.9323868757421496


In [346]:
stacking_regressor = StackingRegressionSolver([svm, linreg], DecisionTreeRegressor())
stacking_regressor.fit(X_train, y_train)
y_pred_test = stacking_regressor.predict(X_test)
y_pred_train = stacking_regressor.predict(X_train)
draw_metrics(y_pred_test, y_pred_train, y_test, y_train)

c:\Users\Дмитрий\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Test
MSE & RMSE & R2
99115.12759977931 314.82555105928 -23.871295032725293
Train
MSE & RMSE & R2
91.84929225422567 9.583803642303282 0.9323868757421496


In [347]:
stacking_regressor = StackingRegressionSolver([tree], LinearRegression())
stacking_regressor.fit(X_train, y_train)
y_pred_test = stacking_regressor.predict(X_test)
y_pred_train = stacking_regressor.predict(X_train)
draw_metrics(y_pred_test, y_pred_train, y_test, y_train)

Test
MSE & RMSE & R2
3882.6375607858618 62.31081415601839 0.025716592230255575
Train
MSE & RMSE & R2
91.84929225422567 9.583803642303282 0.9323868757421496


In [348]:
stacking_regressor = StackingRegressionSolver([svm, LinearRegression()], LinearRegression())
stacking_regressor.fit(X_train, y_train)
y_pred_test = stacking_regressor.predict(X_test)
y_pred_train = stacking_regressor.predict(X_train)
draw_metrics(y_pred_test, y_pred_train, y_test, y_train)

c:\Users\Дмитрий\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Test
MSE & RMSE & R2
3.8532945356168e+22 196298103292.33444 -9.669202629723195e+18
Train
MSE & RMSE & R2
586.8134234767188 24.22423215453317 0.5680283653369075


In [349]:
stacking_regressor = StackingRegressionSolver([linreg], tree)
stacking_regressor.fit(X_train, y_train)
y_pred_test = stacking_regressor.predict(X_test)
y_pred_train = stacking_regressor.predict(X_train)
draw_metrics(y_pred_test, y_pred_train, y_test, y_train)

Test
MSE & RMSE & R2
99260.08859828088 315.05569126470465 -23.90767058759043
Train
MSE & RMSE & R2
96.5819859238835 9.827613439888827 0.928902992553641


In [350]:
stacking_regressor = StackingRegressionSolver([svm, tree, linreg], LinearRegression())
stacking_regressor.fit(X_train, y_train)
y_pred_test = stacking_regressor.predict(X_test)
y_pred_train = stacking_regressor.predict(X_train)
draw_metrics(y_pred_test, y_pred_train, y_test, y_train)

c:\Users\Дмитрий\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Test
MSE & RMSE & R2
3920.250539590752 62.611904136439996 0.016278239926523552
Train
MSE & RMSE & R2
91.8492922542257 9.583803642303284 0.9323868757421496


Самый лучший вариант получился с базой: SVM, Tree(SVM, LinReg) и Метамоделью: LinReg(LinReg) . Давайте попробуем перебрать параметры

In [351]:
linreg = LinearRegression()
svm = LinearSVR()
tree = DecisionTreeRegressor(max_depth=5, min_samples_split=300, min_samples_leaf=5)

In [352]:
stacking_regressor = StackingRegressionSolver([svm, LinearRegression()], LinearRegression())
stacking_regressor.fit(X_train, y_train)
y_pred_test = stacking_regressor.predict(X_test)
y_pred_train = stacking_regressor.predict(X_train)
draw_metrics(y_pred_test, y_pred_train, y_test, y_train)

Test
MSE & RMSE & R2
3.8532952323748776e+22 196298121039.78168 -9.669204378121865e+18
Train
MSE & RMSE & R2
586.8134234822063 24.224232154646437 0.5680283653328679


In [353]:
stacking_regressor = StackingRegressionSolver([svm, tree], linreg)
stacking_regressor.fit(X_train, y_train)
y_pred = stacking_regressor.predict(X_test)
y_pred_test = stacking_regressor.predict(X_test)
y_pred_train = stacking_regressor.predict(X_train)
draw_metrics(y_pred_test, y_pred_train, y_test, y_train)

c:\Users\Дмитрий\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Test
MSE & RMSE & R2
3726.6633890360044 61.04640357167656 0.06485572000033679
Train
MSE & RMSE & R2
1070.7447290176099 32.722236002718546 0.21179146148319583


(SVM, Linear) + (Linear) - очень сильно переобучается в сравнении с моделькой (SVM, tree) + (Linear), хотя последняя даже показывает лучше результаты

## Использование встроенной модели стекинга (0.5 балла)

In [354]:
from sklearn.ensemble import StackingRegressor

estimators = [('svm', SVR()),
              ('tree', DecisionTreeRegressor(max_depth=5, min_samples_split=500))]

stacking_regressor = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
stacking_regressor.fit(X_train, y_train)
y_pred_test = stacking_regressor.predict(X_test)
y_pred_train = stacking_regressor.predict(X_train)
draw_metrics(y_pred_test, y_pred_train, y_test, y_train)

c:\Users\Дмитрий\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:402: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


Test
MSE & RMSE & R2
3778.346975317163 61.46825990149032 0.051886582454164
Train
MSE & RMSE & R2
1084.764667006218 32.93576577227586 0.2014709485424171


c:\Users\Дмитрий\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:402: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


## Блендинг (0.5 балла)

Реализуйте схему блендинга. Для этого разбейте **тестовую** выборку на *валидационную* и *тестовую* части, при необходимости также доработайте код класса `StackingRegressionSolver`. Используйте для обучения базовых моделей обучающую выборку, а для обучения метамодели - валидационную.

Как изменилось качество? Как вы думаете, правдоподобнее ли выглядит такой результат?

In [355]:
class StackingRegressionSolver:
    def __init__(self, base_estimators: list, meta_estimator):
        self._base_estimators = base_estimators
        self._meta_estimator = meta_estimator

    def _fit_base(self, X: pd.DataFrame, y: pd.Series) -> None:
        for estimator in self._base_estimators:
            estimator.fit(X, y)

    def _predict_base(self, X: pd.DataFrame) -> pd.DataFrame:
        return np.array([estimator.predict(X) for estimator in self._base_estimators]).T

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series, X_val: pd.DataFrame, y_val: pd.Series):
        self._fit_base(X_train, y_train)
        meta_features = self._predict_base(X_val)
        self._meta_estimator.fit(meta_features, y_val)
        return self

    def predict(self, X: pd.DataFrame) -> pd.Series:
        meta_features = self._predict_base(X)
        return self._meta_estimator.predict(meta_features)

In [357]:
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(transform_output="pandas")

path = "C:/Users/Дмитрий/Desktop/hw_10/ml-fall22/10. Ensembles/prepaired_data.csv"
prepaired_data = pd.read_csv(path)
X_train, X_test = prepaired_data.query('split=="train"'), prepaired_data.query('split=="test"')
y_train, y_test = prepaired_data.query('split=="train"').target, prepaired_data.query('split=="test"').target

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.5, random_state=42)

trans = ColumnTransformer(
    [('num', StandardScaler(with_mean=False), [1,2,3,4]),
     ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), [0,5,6,7,8,9])])
     
X_train= trans.fit_transform(X_train)
X_val = trans.transform(X_val)
X_test = trans.transform(X_test)

In [358]:
linreg = LinearRegression()
svm = LinearSVR()
tree = DecisionTreeRegressor(max_depth=4, min_samples_split=300, min_samples_leaf=5)

stacking_regressor = StackingRegressionSolver([svm, tree], linreg)
stacking_regressor.fit(X_train, y_train, X_val, y_val)
y_pred = stacking_regressor.predict(X_test)
y_pred_test = stacking_regressor.predict(X_test)
y_pred_train = stacking_regressor.predict(X_train)
draw_metrics(y_pred_test, y_pred_train, y_test, y_train)

Test
MSE & RMSE & R2
3800.665244650831 61.64953564018817 0.04628618345696067
Train
MSE & RMSE & R2
1114.008160260092 33.376760781419335 0.20126849250567957


Качество незначительно лучше. Думаю, что такой результат более реальный, потому что валидация это более независимая методика.

# Бэггинг (максимум 3 балла)

В этой секции аналогично нужно решить задачу при помощи бэггинга - сначала написанного самостоятельно, а затем взятого из sklearn.

## Бэггинг своими руками (2 балла)

Решите задачу, используя в качестве базовой модели линейную регрессию, дерево и SVM. Какой из алгоритмов в качестве базовой модели дает лучший результат? Почему, как вы думаете?

In [359]:
from typing import Tuple
import random

class BaggingRegressionSolver:
    def __init__(
        self,
        base_estimator_ctor,
        max_samples: float = 1,
        n_estimators: int = 10,
        sample_random_state=42,
        **model_kwargs
    ):
        if max_samples < 0 or max_samples > 1:
            raise ValueError
        self._estimators = [
            base_estimator_ctor(**model_kwargs) for _ in range(n_estimators)
        ]
        self._max_samples = max_samples
        self._random_state = sample_random_state

    def _sample_data(self, X: pd.DataFrame, y: pd.Series) -> Tuple[pd.DataFrame, pd.Series]:
        x_i = X.sample(frac=self._max_samples,)
        y_i = y.loc[x_i.index]
        return x_i, y_i

    def fit(self, X: pd.DataFrame, y: pd.Series):
        for estimator in self._estimators:
            x_i, y_i = self._sample_data(X, y)
            estimator.fit(x_i, y_i)

    def predict(self, X: pd.DataFrame) -> pd.Series:
        return np.median([estimator.predict(X) for estimator in self._estimators], axis=0)

In [360]:
from sklearn import set_config

path = "C:/Users/Дмитрий/Desktop/hw_10/ml-fall22/10. Ensembles/prepaired_data.csv"
prepaired_data = pd.read_csv(path)
X_train, X_test = prepaired_data.query('split=="train"'), prepaired_data.query('split=="test"')
y_train, y_test = prepaired_data.query('split=="train"').target, prepaired_data.query('split=="test"').target

trans = ColumnTransformer(
    [('num', StandardScaler(with_mean=False), [1,2,3,4]),
     ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), [0,5,6,7,8,9])])

X_train= trans.fit_transform(X_train)
X_test = trans.transform(X_test)

In [361]:
base_models = [LinearRegression, DecisionTreeRegressor, LinearSVR]

for base_model in base_models:
    
    bagging_Regression = BaggingRegressionSolver(base_model)
    bagging_Regression.fit(X_train, y_train)
    y_pred_test = bagging_Regression.predict(X_test)
    y_pred_train = bagging_Regression.predict(X_train)
    draw_metrics(y_pred_test, y_pred_train, y_test, y_train)

Test
MSE & RMSE & R2
7.391234064315752e+21 85972286606.29977 -1.8547074248021816e+18
Train
MSE & RMSE & R2
586.8086990702634 24.224134640276905 0.5680318431196195
Test
MSE & RMSE & R2
3890.147267546314 62.37104510545189 0.02383215605011313
Train
MSE & RMSE & R2
91.84929225422567 9.583803642303282 0.9323868757421496


c:\Users\Дмитрий\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Дмитрий\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Дмитрий\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Дмитрий\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Дмитрий\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iteration

Test
MSE & RMSE & R2
3920.783493931441 62.61616000627507 0.016144503886755923
Train
MSE & RMSE & R2
1233.9571958867957 35.1277268818635 0.09164568210895307


SVM показал себя лучше всех. Переобучился меньше, чем деревья. Про линейную регрессию вообще молчу))

## Использование встроенной модели бэггинга (1 балл)

Решите задачу, используя:
- `sklearn.ensemble.BaggingRegressor`. В качестве базовой модели попробуйте линейную регрессию, дерево и SVM
- `sklearn.ensemble.RandomForestRegressor`

Какая модель дает лучший результат? Попробуйте улучшить его, перебрав несколько гиперпараметров (как у базовой модели, так и у ансамбля).



In [ ]:
from sklearn.ensemble import StackingRegressor, BaggingRegressor, RandomForestRegressor

base_models = [("LinRegr", LinearRegression()), ('Tree', DecisionTreeRegressor()), ('SVM', LinearSVR())]

for name, model in base_models:
    bagging_Regression = BaggingRegressor(model)
    bagging_Regression.fit(X_train, y_train)
    print(f'Base: {name}')
    y_pred_test = bagging_Regression.predict(X_test)
    y_pred_train = bagging_Regression.predict(X_train)
    draw_metrics(y_pred_test, y_pred_train, y_test, y_train)

Base: LinRegr
Test
3.757319227817121e+23 612969756824.6838 -9.428368535680444e+19
Train
4.7021861075999215e+23 685724879787.7996 -3.461425622058233e+20

Base: Tree
Test
3770.171785330131 61.40172461201828 0.053938010596768726
Train
286.4915406279927 16.926060989728022 0.7891046554644269



c:\Users\Дмитрий\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Дмитрий\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Дмитрий\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Дмитрий\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Дмитрий\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iteration

Base: SVM
Test
3914.5447899931232 62.56632312988452 0.01771000301922221
Train
1224.3260815924152 34.99037126971383 0.09873544525840772



In [362]:
RandomForestRegressorion = RandomForestRegressor()
RandomForestRegressorion.fit(X_train, y_train)
y_pred_test = RandomForestRegressorion.predict(X_test)
y_pred_train = RandomForestRegressorion.predict(X_train)
draw_metrics(y_pred_test, y_pred_train, y_test, y_train)

Test
MSE & RMSE & R2
3718.1378635336196 60.976535351999296 0.06699505902167213
Train
MSE & RMSE & R2
226.72217524188144 15.057296412101392 0.8331027465708729


ООООО, это то, что надо. Давайте попробуем параметры перебрать

In [334]:
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth' : [None] + [1, 20],
    "max_features": np.linspace(0.5, 1, 2),
    'max_samples' : np.linspace(0.5, 1, 2),
    'n_estimators' : [1, 200]
}

search_params = GridSearchCV(RandomForestRegressorion, params, scoring="r2")
search_params.fit(X_train, y_train)
print(f"Best params: {search_params.best_params_}")
y_pred_test = search_params.predict(X_test)
y_pred_train = search_params.predict(X_train)
draw_metrics(y_pred_test, y_pred_train, y_test, y_train)

Best params: {'max_depth': None, 'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 200}
Test
3706.8078753069944 60.883559975637056 0.06983813138333572
Train
440.19634815350616 20.98085670685318 0.6759577602059781


# Бустинг (максимум 3 балла)

## Бустинг своими руками (2 балла)

Решите задачу при помощи алгоритма бустинга, используя в качестве базовой модели:
- Линейную регрессию
- Дерево
- Случайный лес

Какая модель дает лучший результат? Попробуйте улучшить его, перебрав несколько гиперпараметров (как у базовой модели, так и у ансамбля).

In [377]:
from abc import ABC, abstractmethod
from collections import deque
from typing import Tuple

from sklearn.dummy import DummyRegressor


class Loss(ABC):
    """
    Базовый класс для функции потерь
    """
    @abstractmethod
    def forward(self, y_true: pd.Series, y_pred: pd.Series) -> float:
        """
        Метод, вычисляющий значение функции потерь
        """
        pass

    @abstractmethod
    def backward(self, y_true: pd.Series, y_pred: pd.Series) -> pd.Series:
        """
        Метод, вычисляющий значение градиента функции потерь по предсказаниям модели
        """
        pass


class MSELoss(Loss): 
    def forward(self, y_pred: pd.Series, y_true: pd.Series) -> float:  # посчитаем значение ошибки
        return ((y_pred - y_true) ** 2).mean()

    def backward(self, y_pred: pd.Series, y_true: pd.Series) -> pd.Series:  # посчитаем производную по выходам модели
        return y_true - y_pred


class GradientBoostingRegressionSolver:
    def __init__(
        self,
        base_estimator_ctor,
        n_estimators: int = 10,
        loss: Loss = MSELoss(),
        learning_rate: float = 0.1,
        early_stopping: int = 5,
        **model_kwargs
    ):
        if early_stopping < 0:
            raise ValueError

        self._ctor = base_estimator_ctor
        self._kwargs = model_kwargs
        self._n_estimators = n_estimators
        self._estimators = []
        self._early_stopping = early_stopping
        self._loss = loss
        self._lr = learning_rate
        self._random_state = 42

    def _sample_data(self, X: pd.DataFrame, y: pd.Series, frac: float) -> Tuple[pd.DataFrame, pd.Series]:
        x_sample = X.sample(frac=frac, random_state=self._random_state)
        y_sample = y.loc[x_sample.index]
        return x_sample, y_sample

    def _split_data(self, X: pd.DataFrame, y: pd.Series, val_size: float) -> Tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
        x_val, y_val = self._sample_data(X, y, val_size)
        x_train, y_train = X[~X.index.isin(x_val.index)], y[~y.index.isin(y_val.index)]
        return x_train, x_val, y_train, y_val

    def predict(self, X: pd.DataFrame) -> pd.Series:
        return np.sum([estimator.predict(X) for estimator in self._estimators], axis=0)
        
    def fit(self, X: pd.DataFrame, y: pd.Series, val_size: float = 0.1):
        x_train, x_val, y_train, y_val = self._split_data(X, y, val_size)  # Хотим получить валидационную выборку, не тратя на это время снаружи
        base_estimator = DummyRegressor()  # Создадим и обучим базовую модель
        base_estimator.fit(x_train, y_train)
        self._estimators.append(base_estimator)  # Добавим базовую модель в список моделей

        y_pred_train, y_pred_val = base_estimator.predict(x_train), base_estimator.predict(x_val)  # Посчитаем предсказания как на обучающей, так и на валидационной выборках
        train_loss, val_loss = self._loss.forward(y_train, y_pred_train), self._loss.forward(y_val, y_pred_val)  # Посчитаем значение функции потерь для обучения и валидации
        balance = -self._lr * self._loss.backward(y_train, y_pred_train)  # Посчитаем остатки, используя градиент функции потерь
        # print(f'train loss: {train_loss}, val loss: {val_loss}')

        prev_val_loss = val_loss
        k = 0
        for i in range(self._n_estimators - 1):
            estimator = self._ctor(**self._kwargs)  # Создадим очередную модель
            # 1. Обучим её и добавим в список моделей
            estimator.fit(x_train, balance)
            self._estimators.append(estimator)
            # 2. Предскажем ВСЕМ ансамблем данные из обучающей выборки, то же самое сделаем для валидационной
            y_pred_train, y_pred_val = self.predict(x_train), self.predict(x_val)
            # 3. Посчитаем значения функции потерь (на обучении и валидации)
            train_loss, val_loss = self._loss.forward(y_train, y_pred_train), self._loss.forward(y_val, y_pred_val)
            # 4. Обновим остатки для обучающей выборки
            balance = -self._lr * self._loss.backward(y_train, y_pred_train)
            # print(f'train loss: {train_loss}, val loss: {val_loss}')
            # Если валидационный лосс несколько (self._early_stopping) шагов подряд не уменьшается, то остановим обучение
            if val_loss >= prev_val_loss:
                k += 1
            if k >= self._early_stopping:
                break
            prev_val_loss = val_loss

Вопросы на дополнительный балл:
- Почему градиент по ответам мы берем со знаком минус?
- Почему в обучении мы домножаем на `learning_rate`, а в предсказаниях этого не делаем?

Мы же минимизируем функцию, а не максимизируем. Нам же нужно найти минимум, а не максимум. Именно поэтому мы градиент по ответам берём со знаком минус. Градиент же по дефолту указывает нам на скореейшее возрастание функции, но если мы этот вектор умножим на (-1), то получим как раз то, что нам нужно. 

Learning_rate - в самом названии таится вся разгадка! Он указывает нам на шаг, при котором модель учится, чтобы найти свой минимум. Если он будет большой - мы его проскочим, если он будет слишком маленьким - будем долго до него идти. А зачем нам это делать в предсказаниях? Мы уже обучили модель, нашли всё, что нам нужно на наших данных. 

In [378]:
base_models = [LinearRegression, DecisionTreeRegressor, RandomForestRegressor]

for base_model in base_models:
    model = GradientBoostingRegressionSolver(base_model)
    model.fit(X_train, y_train)
    print(base_model.__name__)
    y_pred_test = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    draw_metrics(y_pred_test, y_pred_train, y_test, y_train)

LinearRegression
Test
MSE & RMSE & R2
1.1496962966714456e+23 339071717586.62585 -2.8849718993461555e+19
Train
MSE & RMSE & R2
3.2715562729088997e+22 180874439125.84497 -2.4082944502663033e+19
DecisionTreeRegressor
Test
MSE & RMSE & R2
3729.2147492278036 61.06729688816923 0.06421549853665443
Train
MSE & RMSE & R2
344.5039652297996 18.560818010793586 0.7463999031812574
RandomForestRegressor
Test
MSE & RMSE & R2
3707.370833724175 60.88818303845316 0.06969686631892102
Train
MSE & RMSE & R2
466.3364070681017 21.594823617434383 0.6567152488253989


# Catboost (1 балл)

Решите эту же задачу при помощи `catboost`, не перебирая гиперпараметры. Насколько лучше или хуже справился катбуст? В качестве эксперимента также попробуйте закинуть в него данные без предобработки (разумеется, выкинув ненужные колонки). Изменилось ли качество? Каким образом?

In [371]:
import catboost

model = catboost.CatBoostRegressor()

model.fit(X_train, y_train)

y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)
draw_metrics(y_pred_test, y_pred_train, y_test, y_train)

Learning rate set to 0.054298
0:	learn: 36.6270981	total: 6.18ms	remaining: 6.18s
1:	learn: 36.4101270	total: 13ms	remaining: 6.48s
2:	learn: 36.2219716	total: 19.6ms	remaining: 6.53s
3:	learn: 36.0025654	total: 25.9ms	remaining: 6.45s
4:	learn: 35.8149619	total: 32.3ms	remaining: 6.42s
5:	learn: 35.7441449	total: 38.9ms	remaining: 6.45s
6:	learn: 35.5506920	total: 45.3ms	remaining: 6.42s
7:	learn: 35.4105632	total: 51.8ms	remaining: 6.42s
8:	learn: 35.3111169	total: 58.7ms	remaining: 6.46s
9:	learn: 35.2391904	total: 65ms	remaining: 6.44s
10:	learn: 35.1176114	total: 71.3ms	remaining: 6.41s
11:	learn: 34.9984703	total: 77.6ms	remaining: 6.39s
12:	learn: 34.9218486	total: 84.2ms	remaining: 6.4s
13:	learn: 34.8429712	total: 90.7ms	remaining: 6.39s
14:	learn: 34.7080114	total: 97ms	remaining: 6.37s
15:	learn: 34.6480272	total: 103ms	remaining: 6.36s
16:	learn: 34.4959077	total: 110ms	remaining: 6.35s
17:	learn: 34.4172237	total: 116ms	remaining: 6.32s
18:	learn: 34.3459168	total: 122ms	r

Catboost справился пока что лучше всех. Давайте попробуем ему скормить сырые данные без преодобработки.

Не совсем вас понял, что такое без преобработки. Если совсем без - она показывает максимально плохой результат, если с предобработкой, то она показывает вполне хороший результат. 